As a capstone to this fundamentals course, prepare an Analytic Report and Research Proposal on a dataset of your choosing. Your Report should accomplish these three goals:

    Describe your dataset. Describe and explore your dataset in the initial section of your Report. What does your data contain and what is its background? Where does it come from? Why is it interesting or significant? Conduct summary statistics and produce visualizations for the particular variables from the dataset that you will use.

    Ask and answer analytic questions. Ask three analytic questions and answer each one with a combination of statistics and visualizations. These analytic questions can focus on individuals behaviors or comparisons of the population.

    Propose further research. Lastly, make a proposal for a realistic future research project on this dataset that would use some data science techniques you'd like to learn in the bootcamp. Just like your earlier questions, your research proposal should present one or more clear questions. Then you should describe the techniques you would apply in order to arrive at an answer.

Report guidelines

Keep these guidelines in mind as you draft your Report:

    Length. Your Report should be three to five pages long with visualizations. Short and clear is better than long and opaque.
    Structure. Pay attention to the narrative structure of your Report. Each section should flow into the next and make a logical, readable text. Don't simply create a list of bullet points or present code without explanation. 
    
    
    
    http://insideairbnb.com/get-the-data.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

%matplotlib inline

In [2]:
df = pd.read_csv('airbnb_listings.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,7893,North Williams 2brm Home,21489,Jesse,NaN,Piedmont,45.575479,-122.666777,Entire home/apt,175,3,2,8/12/2014,0.02,1,35
1,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",49682,Ali And David,NaN,Concordia,45.564010,-122.634717,Private room,65,2,500,10/21/2018,4.67,1,303
2,16688,"Beautiful condo in downtown Pearl, NW Portland",64840,Ashish,NaN,Pearl,45.525418,-122.685566,Entire home/apt,275,2,44,11/4/2018,2.37,1,340
3,25200,Flamenco Dream :: hip mississippi ave.,104038,Garden,NaN,Humboldt,45.556750,-122.677839,Private room,49,7,34,10/18/2018,0.33,6,180
4,26203,Bluebird @ hip mississippi ave,104038,Garden,NaN,Humboldt,45.556965,-122.675656,Private room,49,3,28,9/30/2018,0.27,6,167


In [3]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,5.317000e+03,5.317000e+03,0.0,5317.000000,5317.000000,5317.000000,5317.000000,5317.000000,4877.000000,5317.000000,5317.000000
mean,1.531575e+07,4.244948e+07,NaN,45.527711,-122.648023,117.008275,4.221930,60.934173,3.013443,2.123190,144.244123
std,8.481055e+06,5.331177e+07,NaN,0.033481,0.046692,156.226618,10.049577,81.961679,2.920180,4.193996,127.974824
min,7.893000e+03,2.531000e+03,NaN,45.433945,-122.833981,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,7.959145e+06,5.192007e+06,NaN,45.504863,-122.680211,65.000000,1.000000,6.000000,0.780000,1.000000,22.000000
50%,1.625721e+07,1.926291e+07,NaN,45.526658,-122.649032,90.000000,2.000000,29.000000,2.380000,1.000000,115.000000
75%,2.188217e+07,5.560689e+07,NaN,45.555758,-122.621381,129.000000,2.000000,84.000000,4.450000,2.000000,268.000000
max,2.986167e+07,2.240317e+08,NaN,45.611885,-122.483763,8400.000000,222.000000,865.000000,58.160000,41.000000,365.000000


In [5]:
df = df.drop(columns=['neighbourhood_group', 'name', 'last_review'])

In [6]:
df.head()

,id,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,7893,21489,Jesse,Piedmont,45.575479,-122.666777,Entire home/apt,175,3,2,0.02,1,35
1,12899,49682,Ali And David,Concordia,45.564010,-122.634717,Private room,65,2,500,4.67,1,303
2,16688,64840,Ashish,Pearl,45.525418,-122.685566,Entire home/apt,275,2,44,2.37,1,340
3,25200,104038,Garden,Humboldt,45.556750,-122.677839,Private room,49,7,34,0.33,6,180
4,26203,104038,Garden,Humboldt,45.556965,-122.675656,Private room,49,3,28,0.27,6,167


theme: report for someone wanting to build a side income renting out airbnb
-  could consider looking at monthly rental prices in the area as well, to see if airbnb is more profitable than renting, data here: https://www.apartmentlist.com/rentonomics/rental-price-data/
1. how much can one expect to make renting vs airbnb?
2. how much does it cost to pay off an apartment/house, how long would it take to pay off property with airbnb rentals?
3. links between: neighborhood and price (are room types confounding?), number of reviews//reviews per month an price, room type and price, availability over the course of a year and price. aka how can one maximize their profits on airbnb?